# 22.02.21 과제 전 Method1 df 정리

# Library

In [2]:
#데이터 분석 라이브러리
import numpy as np
import pandas as pd

#딥러닝 라이브러리
import tensorflow as tf

#결과 확인을 용이하게 하기 위한 코드
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

numpy version: 1.19.5
matplotlib version: 3.4.2
seaborn version: 0.11.1
--------------------------------------------------


# GPU

In [53]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:  # gpu가 있다면, 용량 한도를 5GB로 설정
    tf.config.experimental.set_virtual_device_configuration(gpus[0], 
                                                            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5*1024)])

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


# CPU

In [54]:
#GPU 사용 설정, -1이면 CPU 사용
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# Data Load

In [115]:
all_to_1 = pd.read_csv('df_all_best_for_meth1_preds2.csv') # 0 to 1 (all to 1) modeling
# preds = pd.read_csv('df_all_preds_for_meth1_preds2.csv') # 할 필요 없음.
inversed = pd.read_csv('/project/LSH/df_all_best.csv') # inverse modeling
Method1 = pd.read_csv('Method1_Entropy.csv')

In [116]:
relu_and_sign = pd.read_csv('Method1.csv')

# sort data by feature

## relu_and_sign sorting

In [117]:
relu_and_sign.sort_values('feature', inplace = True)
relu_and_sign.reset_index(inplace = True)

relu_and_sign.drop('index', axis = 1, inplace = True)

In [118]:
relu_and_sign = relu_and_sign[['feature', 'diff', 'diff_preds']]

# df colnames 바꾸는 2가지 방법
# relu_and_sign.columns = ['feature', 'ReLU', 'Sign'] # [['feature', 'ReLU', 'Sign']]여도 됨.
relu_and_sign.rename(columns = {'diff':'ReLU', 'diff_preds':'Sign'}, inplace = True)

In [119]:
# relu_and_sign

,feature,ReLU,Sign
0,0,0.000000,-1.0
1,50803,0.002118,-1.0
2,50804,0.003495,1.0
3,50805,0.001575,-1.0
4,50806,0.005405,1.0
...,...,...,...
4063,78112001103,0.002014,-1.0
4064,79511050204,0.000000,-1.0
4065,87701071218,0.013409,1.0
4066,87701083336,0.000000,1.0


## all_to_1 sorting

In [120]:
all_to_1 = all_to_1[['feature', 'preds_mean', 'preds2_mean', 'diff_preds']]

all_to_1.sort_values('feature', inplace = True)
all_to_1.reset_index(inplace = True)

all_to_1.drop('index', axis = 1, inplace = True)

## inversed sorting

In [121]:
inversed.sort_values('feature', inplace = True)
inversed.reset_index(inplace = True)
inversed.drop('index', axis = 1, inplace = True)

## Entropy
(이미 x data가 feature 별로 sorting 돼 있는 상태)

In [122]:
# Entropy 함수
def H(p):
    id_p = np.where(p != 0)
    return -np.sum(p[id_p] * np.log2(p[id_p]))

In [123]:
# data load
x = np.load('/project/LSH/x_(7727,10,4068).npy')

# data processing
x_sum = x.sum(axis = 1) # time 죽이기
# x_sum.shape
# x_sum
x_item_only = x_sum.sum(axis = 0) # sub 죽이기
# x_item_only.shape
# x_item_only

# Entropy에 쓸 비율 구하기
sub_time = 7727*10
ratio1_lst = []
for i in x_item_only:
    ratio1 = i / sub_time
    ratio1_lst.append(ratio1)
# ratio1_lst

# list to array
ratio1_arr = np.array(ratio1_lst)
ratio0_arr = 1 - ratio1_arr

In [124]:
# Entropy 함수에 적용
Entropy_lst = []
for i in range(len(ratio0_arr)):
    input_arr = np.array([ratio0_arr[i], ratio1_arr[i]]) # input arr 생성
    en = H(input_arr)
    Entropy_lst.append(en)
# Entropy_lst

# list to array
Entropy_arr = np.array(Entropy_lst)

In [125]:
Entropy_sr = pd.Series(Entropy_arr, name = 'Entropy')

## Method1 sorting

In [126]:
Method1.sort_values('feature', inplace = True)
Method1.reset_index(inplace = True)
Method1.drop('index', axis = 1, inplace = True)

In [127]:
all_to_1
inversed
Method1

,feature,preds_mean,preds2_mean,diff_preds
0,0,0.633561,0.613608,-0.019953
1,50803,0.633561,0.607474,-0.026087
2,50804,0.633561,0.666301,0.032740
3,50805,0.633561,0.599635,-0.033926
4,50806,0.633561,0.682195,0.048633
...,...,...,...,...
4063,78112001103,0.633561,0.583690,-0.049871
4064,79511050204,0.633561,0.631427,-0.002134
4065,87701071218,0.633561,0.722150,0.088589
4066,87701083336,0.633561,0.640804,0.007243


,feature,baseline_bce,bce,diff
0,0,1.014362,1.009931,-0.004431
1,50803,1.014362,1.016480,0.002118
2,50804,1.014362,1.017856,0.003495
3,50805,1.014362,1.015937,0.001575
4,50806,1.014362,1.019767,0.005405
...,...,...,...,...
4063,78112001103,1.014362,1.016376,0.002014
4064,79511050204,1.014362,1.014257,-0.000105
4065,87701071218,1.014362,1.027771,0.013409
4066,87701083336,1.014362,1.014223,-0.000139


,feature,Method1
0,0,-0.000000
1,50803,-0.000071
2,50804,0.001694
3,50805,-0.000002
4,50806,0.000404
...,...,...
4063,78112001103,-0.000074
4064,79511050204,0.000000
4065,87701071218,0.000766
4066,87701083336,0.000000


# Concat Data

In [128]:
concat_data = pd.concat([inversed, relu_and_sign[['ReLU']], all_to_1[['preds_mean', 'preds2_mean', 'diff_preds']], relu_and_sign[['Sign']], Entropy_sr, Method1['Method1']], axis = 1)

In [129]:
concat_data

,feature,baseline_bce,bce,diff,ReLU,preds_mean,preds2_mean,diff_preds,Sign,Entropy,Method1
0,0,1.014362,1.009931,-0.004431,0.000000,0.633561,0.613608,-0.019953,-1.0,0.894548,-0.000000
1,50803,1.014362,1.016480,0.002118,0.002118,0.633561,0.607474,-0.026087,-1.0,0.033443,-0.000071
2,50804,1.014362,1.017856,0.003495,0.003495,0.633561,0.666301,0.032740,1.0,0.484874,0.001694
3,50805,1.014362,1.015937,0.001575,0.001575,0.633561,0.599635,-0.033926,-1.0,0.001172,-0.000002
4,50806,1.014362,1.019767,0.005405,0.005405,0.633561,0.682195,0.048633,1.0,0.074665,0.000404
...,...,...,...,...,...,...,...,...,...,...,...
4063,78112001103,1.014362,1.016376,0.002014,0.002014,0.633561,0.583690,-0.049871,-1.0,0.036891,-0.000074
4064,79511050204,1.014362,1.014257,-0.000105,0.000000,0.633561,0.631427,-0.002134,-1.0,-0.000000,0.000000
4065,87701071218,1.014362,1.027771,0.013409,0.013409,0.633561,0.722150,0.088589,1.0,0.057111,0.000766
4066,87701083336,1.014362,1.014223,-0.000139,0.000000,0.633561,0.640804,0.007243,1.0,0.011373,0.000000


In [130]:
# concat_data.to_csv('Method1_concat_all.csv', index = False)

# Concat Data Load

In [131]:
data = pd.read_csv('Method1_concat_all.csv')

In [132]:
data
data.info()
data.describe()

,feature,baseline_bce,bce,diff,ReLU,preds_mean,preds2_mean,diff_preds,Sign,Entropy,Method1
0,0,1.014362,1.009931,-0.004431,0.000000,0.633561,0.613608,-0.019953,-1.0,0.894548,-0.000000
1,50803,1.014362,1.016480,0.002118,0.002118,0.633561,0.607474,-0.026087,-1.0,0.033443,-0.000071
2,50804,1.014362,1.017856,0.003495,0.003495,0.633561,0.666301,0.032740,1.0,0.484874,0.001694
3,50805,1.014362,1.015937,0.001575,0.001575,0.633561,0.599635,-0.033926,-1.0,0.001172,-0.000002
4,50806,1.014362,1.019767,0.005405,0.005405,0.633561,0.682195,0.048633,1.0,0.074665,0.000404
...,...,...,...,...,...,...,...,...,...,...,...
4063,78112001103,1.014362,1.016376,0.002014,0.002014,0.633561,0.583690,-0.049871,-1.0,0.036891,-0.000074
4064,79511050204,1.014362,1.014257,-0.000105,0.000000,0.633561,0.631427,-0.002134,-1.0,-0.000000,0.000000
4065,87701071218,1.014362,1.027771,0.013409,0.013409,0.633561,0.722150,0.088589,1.0,0.057111,0.000766
4066,87701083336,1.014362,1.014223,-0.000139,0.000000,0.633561,0.640804,0.007243,1.0,0.011373,0.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4068 entries, 0 to 4067
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   feature       4068 non-null   int64  
 1   baseline_bce  4068 non-null   float64
 2   bce           4068 non-null   float64
 3   diff          4068 non-null   float64
 4   ReLU          4068 non-null   float64
 5   preds_mean    4068 non-null   float64
 6   preds2_mean   4068 non-null   float64
 7   diff_preds    4068 non-null   float64
 8   Sign          4068 non-null   float64
 9   Entropy       4068 non-null   float64
 10  Method1       4068 non-null   float64
dtypes: float64(10), int64(1)
memory usage: 349.7 KB


,feature,baseline_bce,bce,diff,ReLU,preds_mean,preds2_mean,diff_preds,Sign,Entropy,Method1
count,4.068000e+03,4068.000000,4068.000000,4068.000000,4068.000000,4068.000000,4068.000000,4068.000000,4068.000000,4068.000000,4068.000000
mean,1.747423e+10,1.014362,1.015917,0.001556,0.001612,0.633561,0.634423,0.000862,0.040806,0.034770,0.000030
std,2.582280e+10,0.000000,0.002900,0.002900,0.002850,0.000000,0.031907,0.031907,0.999290,0.096562,0.000313
min,0.000000e+00,1.014362,1.001220,-0.013142,0.000000,0.633561,0.497052,-0.136509,-1.000000,-0.000000,-0.003352
25%,5.487080e+07,1.014362,1.014395,0.000033,0.000033,0.633561,0.615502,-0.018059,-1.000000,0.000994,-0.000001
50%,3.380929e+08,1.014362,1.014998,0.000636,0.000636,0.633561,0.634036,0.000475,1.000000,0.004665,-0.000000
75%,5.107900e+10,1.014362,1.016333,0.001971,0.001971,0.633561,0.653400,0.019839,1.000000,0.022021,0.000005
max,8.770109e+10,1.014362,1.073542,0.059180,0.059180,0.633561,0.826544,0.192982,1.000000,0.999899,0.005401
